[課題のURL](https://diver.diveintocode.jp/curriculums/1642)

# Sprint 機械学習フロー

(Home Credit Default Risk)[https://www.kaggle.com/c/home-credit-default-risk/overview/evaluation]

# 【問題1】クロスバリデーション
事前学習期間の課題で作成したベースラインモデルに対してKFoldクラスによるクロスバリデーションを行うコードを作成し実行してください。

In [56]:
import time

import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [16]:
df_train = pd.read_csv('application_train.csv')
df_test = pd.read_csv('application_test.csv')

In [17]:
# 欠損値を0で補完
df_train_fillna = df_train.fillna({
                                "EXT_SOURCE_1":0,
                                "EXT_SOURCE_2":0,
                                "EXT_SOURCE_3":0,
                                })

df_test_fillna = df_test.fillna({
                                "EXT_SOURCE_1":0,
                                "EXT_SOURCE_2":0,
                                "EXT_SOURCE_3":0,
                                })

In [21]:
# 特徴量（説明変数）をX、正解（目的変数）をyというndarrayに格納
X = df_train_fillna[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].values
y = df_train_fillna["TARGET"].values

# データを分割
kf = KFold(n_splits=5, shuffle=True, random_state=0 )
# print(kf.split(X))

# 学習
n_count = 0
list_score = []

for train_index, valid_index in kf.split(X):
    
    X_train, X_valid = X[train_index], X[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]
    
    print(n_count+1,"回目")
    
    # 学習
    clf = RandomForestClassifier().fit(X_train, y_train)
    y_vaild_pred = clf.predict_proba(X_valid)
    
    # 評価
    score_temp = roc_auc_score(y_valid, y_vaild_pred[:,1])
    print(f'AUC_test: {score_temp}')
    
    list_score.append(score_temp)
    n_count = n_count + 1

1 回目
AUC_test: 0.6291967189725017
2 回目
AUC_test: 0.6306111722370967
3 回目
AUC_test: 0.6311108603079598
4 回目
AUC_test: 0.6284057005489749
5 回目
AUC_test: 0.6325145316697183


# 【問題2】グリッドサーチ

In [59]:
# 特徴量（説明変数）をX、正解（目的変数）をyというndarrayに格納
X = df_train_fillna[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].values
y = df_train_fillna["TARGET"].values

In [60]:
# X_train（学習用）とX_valid（検証用）に分割
X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y, random_state=0)

In [61]:
clf = RandomForestClassifier().fit(X_train, y_train)

y_vaild_pred = clf.predict_proba(X_valid)

In [62]:
#評価の実行(グリットサーチを使用しない場合)

score_temp = roc_auc_score(y_valid, y_vaild_pred[:,1])
print(f'AUC_test: {score_temp}')

AUC_test: 0.6355082718721963


In [63]:
# グリットサーチの実施
clf = RandomForestClassifier()

params = {
    #"n_estimators": [5, 10, 20, 50, 100], # デフォルトは100
    "max_depth":[1,2,3,4,5,6,7,8,9,10],
    "min_samples_leaf" :[1,2,3,4],
}

grid_search = GridSearchCV(clf,  # 分類器を渡す
                               param_grid=params,  # 試行してほしいパラメータを渡す
                               cv=5,  # 5-Fold CV で汎化性能を調べる
                               )

In [64]:
# グリッドサーチで優れたハイパーパラメータを探す
# 処理前の時刻
t1 = time.time()

grid_search.fit(X_train, y_train)

# 処理後の時刻
t2 = time.time()
# 経過時間を表示
elapsed_time = t2-t1
print(f"経過時間：{elapsed_time}")

経過時間：2610.749603033066


In [65]:
grid_search.best_params_

{'max_depth': 8, 'min_samples_leaf': 1}

In [66]:
# 処理前の時刻
t1 = time.time()

clf = RandomForestClassifier(**grid_search.best_params_).fit(X_train, y_train)

# 処理後の時刻
t2 = time.time()
# 経過時間を表示
elapsed_time = t2-t1
print(f"経過時間：{elapsed_time}")

y_vaild_pred = clf.predict_proba(X_valid)

経過時間：21.453105926513672


In [67]:
#評価の実行(グリットサーチを使用した場合)

score_temp = roc_auc_score(y_valid, y_vaild_pred[:,1])
print(f'AUC_test: {score_temp}')

AUC_test: 0.7261191723854113


# 【問題3】Kaggle Notebooksからの調査

KaggleのNotebooksから様々なアイデアを見つけ出して、列挙してください。

・ハイパーパラメータを探索する際は、グリッドサーチではなくベイズ最適化で実施する。  
→探索速度が速いので実施

・異常値排除  
→異常値はなかった

# 【問題4】高い汎化性能のモデル作成

# 【問題5】最終的なモデルの選定

問題3で見つけたアイデアと、独自のアイデアを組み合わせ高い汎化性能のモデル作りを進めてください。

その過程として、何を行うことで、クロスバリデーションの結果がどの程度変化したかを表にまとめてください。

ランダムフォレストのハイパーパラメータを設定することで下記の様に結果が変化した。

- ハイパーパラメータ：初期値 sScore 0.65456  
- ハイパーパラメータ："max_depth":[1,2,3,4,5,6,7,8,9,10], "min_samples_leaf" :[1,2,3,4],  Score 0.71183  

また、処理速度について下記の様になった。また、精度もほぼ同等となった。
- グリッドサーチ:  2610s, AUC_test: 0.726
- ベイズ最適化: 549s, AUC_test: 0.725

In [107]:
# 特徴量（説明変数）をX、正解（目的変数）をyというndarrayに格納
X = df_train_fillna[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].values
y = df_train_fillna["TARGET"].values

X.shape, y.shape

((307511, 3), (307511,))

In [83]:
upperbound, lowerbound = np.percentile(X, [1, 99])
X = np.clip(X, upperbound, lowerbound)

upperbound, lowerbound = np.percentile(y, [1, 99])
y = np.clip(y, upperbound, lowerbound)

# print(X.shape, y.shape) # ((307511, 3), (307511,)) 外れ値なし

# X_train（学習用）とX_valid（検証用）に分割
X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y, random_state=0)

In [96]:
def randomforest_cv(max_depth, min_samples_leaf):
    # クロスバリデーションを行う
    val = cross_val_score(
        RandomForestClassifier(
            max_depth=int(max_depth),
            min_samples_leaf=int(min_samples_leaf),
        ),
        X_train, y_train, # 学習データ
        scoring='accuracy',
        cv=5,
        n_jobs=-1
    ).mean() #平均値を返す
    return val


In [97]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score

# 探索範囲などの設定
rf_cv_bo = BayesianOptimization(
    randomforest_cv, # 上の関数を与える
    {
        'max_depth':(1,10),
         "min_samples_leaf" :(1,4),
    },
     verbose=2, # 0だと学習過程を表示をしない、デフォルトの2なら全て表示する、1は最高値が更新されたら表示
    random_state=0
)

In [98]:
# 処理前の時刻
t1 = time.time()

rf_cv_bo.maximize(init_points=5, n_iter=10, acq='ei')

# 処理後の時刻
t2 = time.time()
# 経過時間を表示
elapsed_time = t2-t1
print(f"経過時間：{elapsed_time}")

|   iter    |  target   | max_depth | min_sa... |
-------------------------------------------------
|  1        |  0.9193   |  5.939    |  3.146    |
|  2        |  0.9193   |  6.425    |  2.635    |
|  3        |  0.9193   |  4.813    |  2.938    |
|  4        |  0.9193   |  4.938    |  3.675    |
|  5        |  0.9193   |  9.673    |  2.15     |
|  6        |  0.9193   |  1.017    |  1.116    |
|  7        |  0.9193   |  9.942    |  1.136    |
|  8        |  0.9193   |  1.034    |  3.982    |
|  9        |  0.9193   |  9.933    |  1.326    |
|  10       |  0.9193   |  1.007    |  3.965    |
|  11       |  0.9193   |  9.973    |  1.124    |
|  12       |  0.9193   |  1.111    |  3.68     |
|  13       |  0.9193   |  1.063    |  1.039    |
|  14       |  0.9193   |  9.917    |  3.753    |
|  15       |  0.9193   |  1.138    |  3.975    |
経過時間：549.7130920886993


In [99]:
# 実行結果の内、最も精度の良かった評価値とパラメータの値をprint
print(rf_cv_bo.max['target'])
print('max_depth:', rf_cv_bo.max['params']['max_depth'])
print('min_samples_leaf:', rf_cv_bo.max['params']['min_samples_leaf'])

0.9192786811020225
max_depth: 9.672964844509263
min_samples_leaf: 2.150324556477333


In [102]:
# 実行結果の内、最も精度の良かったパラメータで実施
clf = RandomForestClassifier(
    max_depth=int(rf_cv_bo.max['params']['max_depth']),
    min_samples_leaf=int(rf_cv_bo.max['params']['min_samples_leaf']),
    random_state=0
)

In [103]:
clf.fit(X_train, y_train)
y_vaild_pred = clf.predict_proba(X_valid)

In [104]:
#評価の実行(グリットサーチを使用した場合)

score_temp = roc_auc_score(y_valid, y_vaild_pred[:,1])
print(f'AUC_test: {score_temp}')

AUC_test: 0.7251435765475157


In [105]:
# テストデータに対する推定
# 型変換
X_test = df_test_fillna[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].values

# 予測
y_test_pred = clf.predict_proba(X_test)

# 提出用データの作成
# NumPy配列ndarrayからpandasに型変換する
df_score = pd.DataFrame(y_test_pred[:,1], columns=["TARGET"])
#print(df_score)

# 提出フォーマットのデータフレームを作成する
df_sk_id = df_test_fillna["SK_ID_CURR"]
#print(df_sk_id)

df_submission = pd.concat([df_sk_id, df_score], axis=1)
display(df_submission)

# Score 0.71183
df_submission.to_csv('submission.csv', index=False)

,SK_ID_CURR,TARGET
0,100001,0.036335
1,100005,0.065080
2,100013,0.034004
3,100028,0.048793
4,100038,0.138411
...,...,...
48739,456221,0.033772
48740,456222,0.054433
48741,456223,0.048815
48742,456224,0.064060
